In [1]:
# loading libs
import pandas as pd
import numpy as np
import itertools
import warnings
import statsmodels.api as sm
from fbprophet import Prophet
from dateutil import parser
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [2]:
import os
os.chdir('C:\\Users\\dev2\\Desktop\\coo1')
%pwd

'C:\\Users\\dev2\\Desktop\\coo1'

In [14]:
#input
dir_name = "C:\\Users\\dev2\\Desktop\\coo1\\plots"
#Define minimum test and train days
test_points = 2
min_train_days = 731

In [15]:
from datetime import date
from datetime import datetime

import numpy as np
import pandas as pd

import datetime


def iso_year_start(iso_year):
    "The gregorian calendar date of the first day of the given ISO year"
    fourth_jan = datetime.date(iso_year, 1, 4)
    delta = datetime.timedelta(fourth_jan.isoweekday() - 1)
    return fourth_jan - delta


def iso_to_gregorian(iso_year, iso_week, iso_day):
    "Gregorian calendar date for the given ISO year, week and day"
    year_start = iso_year_start(iso_year)
    return year_start + datetime.timedelta(days=iso_day - 1, weeks=iso_week - 1)


def make_date(iso_date_string):
    date_tuple = iso_date_string.split(",")
    iso_year = int(date_tuple[0])
    iso_week = int(date_tuple[1])
    iso_day = int(date_tuple[2])
    return iso_to_gregorian(iso_year=iso_year, iso_week=iso_week, iso_day=iso_day)


def weeks_for_year(year, max_week_present, min_week_present):
    first_week = date(year, 1, 4).isocalendar()
    last_week = date(year, 12, 28).isocalendar()

    if (first_week < min_week_present):
        first_week = min_week_present
        if (max_week_present < last_week):
            last_week = max_week_present
            return (first_week[1], last_week[1])
        else:
            return (first_week[1], last_week[1])
    else:
        if (max_week_present < last_week):
            last_week = max_week_present
            return (first_week[1], last_week[1])
        else:
            return (first_week[1], last_week[1])


def get_missing_weeks(input_year, existing_weeks, max_week_present, min_week_present):
    (_first_week, _total_weeks) = weeks_for_year(input_year, max_week_present, min_week_present)
    _weeks_list = set(range(_first_week, _total_weeks + 1))
    _existing = set(existing_weeks)
    _missing = _weeks_list - _existing
    return list(_missing)


def get_missing_data(df_grpby_year, grp_min_date, grp_max_date):
    final_data_df = pd.DataFrame()
    for year, group in df_grpby_year:
        missing_data_df_per_grp = pd.DataFrame()
        grp_dataset = group.copy()
        grp_dataset['week_num'] = grp_dataset['isocalendar'].map(lambda x: x[1])
        # print grp_dataset
        max_week_present = grp_max_date
        min_week_present = grp_min_date
        _weeks_missed = get_missing_weeks(year, grp_dataset['week_num'], max_week_present, min_week_present)
        _weeks_missed_Series = pd.Series(_weeks_missed).map(str)
        d = pd.DataFrame(np.zeros((len(_weeks_missed), 2)), columns=['quantity', 'q_indep_p'])
        missing_data_df_per_grp = pd.concat([d, _weeks_missed_Series], axis=1)
        missing_data_df_per_grp.columns = ['quantity', 'q_indep_p', 'week_num']
        missing_data_df_per_grp['year'] = str(year)
        missing_data_df_per_grp['isocalendar'] = (
        missing_data_df_per_grp['year'] + "," + missing_data_df_per_grp['week_num'] + "," + "4").map(
            lambda x: make_date(x).isocalendar())

        final_data_df = pd.concat([final_data_df, missing_data_df_per_grp], axis=0)
    # print "printing final_data_df"
    # print final_data_df
    return final_data_df


def get_cmplt_missing_data(raw_data):
    """
    This fucntion receives raw data
    :param raw_data: grouped dataframe
    :return: yet to decide
    """
    raw_data_grp = raw_data.groupby(['customernumber', 'matnr'], as_index=False)
    final_data_df_1 = pd.DataFrame()
    for name, group in raw_data_grp:
        # print name[0]

        final_df_2 = group.copy()
        # final_df_2['date'] = final_df_2['date'].map(str)
        # final_df_2['date_parse'] = pd.to_datetime(final_df_2['date'])
        # final_df_2['isocalendar'] = final_df_2['date_parse'].map(lambda x: x.isocalendar())
        final_df_2['year'] = final_df_2['date_parse'].map(lambda x: x.year)

        group_min_date = final_df_2.ix[final_df_2['date_parse'].argmin()].iloc[6]
        group_max_date = final_df_2.ix[final_df_2['date_parse'].argmax()].iloc[6]

        # print group_min_date
        # print group_max_date
        final_df_2.drop(['date', 'date_parse'], axis=1)
        raw_data_grp_year = final_df_2[
            ['customernumber', 'matnr', 'quantity', 'q_indep_p', 'isocalendar', 'year']].groupby(['year'],
                                                                                                 as_index=False)
        missing_data = get_missing_data(raw_data_grp_year, group_min_date, group_max_date)
        missing_data['matnr'] = name[1]
        missing_data['customernumber'] = name[0]
        # print missing_data
        # print "******************************************************************"
        # final_data_df_2 = pd.concat([final_data_df_2, missing_data], axis=0)
        # print final_data_df_2
        # print "##################################################################"
        final_data_df_1 = pd.concat([final_data_df_1, missing_data], axis=0)

    # print "##################################################################"
    final_data_df_1 = final_data_df_1.drop(['week_num', 'year'], axis=1)
    # print final_data_df_1

    return final_data_df_1

def transform_raw_data(raw_data):
    result_df = pd.DataFrame()
    raw_data_copy = raw_data.copy()
    raw_data_copy['customernumber'] = raw_data_copy['customernumber'].map(str)
    raw_data_copy['date'] = raw_data_copy['date'].map(str)
    raw_data_copy['date_parse'] = pd.to_datetime(raw_data_copy['date'])
    raw_data_copy['isocalendar'] = raw_data_copy['date_parse'].map(lambda x: x.isocalendar())

    missing_Data = get_cmplt_missing_data(raw_data_copy)
    # print "missing data"
    # print missing_Data

    raw_data_copy = raw_data_copy.drop(['date', 'date_parse'], axis=1)
    # print "raw_data_copy"
    # print raw_data_copy

    result_df = pd.concat([raw_data_copy, missing_Data], axis=0)
    # print "result_df"
    # print result_df
    return result_df


def weekly_aggregate(data):
    data['year_weekNum'] = data['isocalendar'].map(lambda x: (x[0], x[1]))
    data = data.drop(['isocalendar'], axis=1)
    data_grp = data.groupby(['customernumber', 'matnr', 'year_weekNum'], as_index=False)[
        ['quantity', 'q_indep_p']].sum()
    data_grp['dt_week'] = data_grp['year_weekNum'].map(lambda x: iso_to_gregorian(int(x[0]), int(x[1]), 4))

    data_grp = data_grp.drop(['year_weekNum'], axis=1)
    # print "data_grp"
    # print data_grp
    return data_grp


def get_weekly_aggregate(inputfile, outputfile, input_sep="\t", output_sep=","):
    raw_data = pd.read_csv(inputfile, sep=input_sep, header=None,
                           names=['customernumber', 'matnr', 'date', 'quantity', 'q_indep_p'])
    # print raw_data
    dataset_cmplt = transform_raw_data(raw_data)
    # print dataset_cmplt
    result = weekly_aggregate(dataset_cmplt)
#     result.to_csv(outputfile, sep=output_sep, index=False)
    return result

In [18]:
def is_outlier(value, p25, p75):
    """Check if value is an outlier
    """
    lower = p25 - 2 * (p75 - p25)
    upper = p75 + 2 * (p75 - p25)
    return value <= lower or value >= upper
 
 
def get_indices_of_outliers(values):
    """Get outlier indices (if any)
    """
    p25 = np.percentile(values, 25)
    p75 = np.percentile(values, 75)
     
    indices_of_outliers = []
    for ind, value in enumerate(values):
        if is_outlier(value, p25, p75):
            indices_of_outliers.append(ind)
    return indices_of_outliers

In [6]:
#data transformation
data = get_weekly_aggregate(inputfile="c001_data_52.txt", outputfile="c001_data_52_agg.txt")
data.dt_week = data.dt_week.apply(str).apply(parser.parse)
data.head()

,customernumber,matnr,quantity,q_indep_p,dt_week
0,500008686,100278,12.0,1.846154,2015-02-05
1,500008686,100278,0.0,0.000000,2015-02-12
2,500008686,100278,0.0,0.000000,2015-02-19
3,500008686,100278,0.0,0.000000,2015-02-26
4,500008686,100278,0.0,0.000000,2015-03-05


In [ ]:
#single product data
for cus_no in data.customernumber.unique():
    cus_data = data.loc[(data['customernumber'] == cus_no)]
    for mat_no in cus_data.matnr.unique():
        prod = cus_data[cus_data.matnr == mat_no]

        prod = prod.sort_values('dt_week')
        prod = prod.reset_index()
        prod = prod.rename(columns={'dt_week': 'ds', 'quantity': 'y'})
        prod = prod[['ds','y']]
        prod = prod.drop(prod.index[[0,len(prod.y)-1]]).reset_index(drop = True)
        # prod['y']=prod['y'].replace(0,prod['y'].mean())
#         prod.dtypes

        # Incremental Test
        train = prod[prod.ds <= (max(prod.ds) - pd.DateOffset(days=(max(prod.ds)-min(prod.ds)).days-min_train_days))]
        test = prod[(max(train.index)+1):(max(train.index)+1+test_points)]
        rem_data = prod[(max(train.index)+test_points):]
        output_result = pd.DataFrame()

        indices_of_outliers = get_indices_of_outliers(prod.y)

        fig = plt.figure()
#         ax = fig.add_subplot(111)
        plt.plot(prod.ds,prod.y, 'b-', label='Quantity')
        plt.plot(
            prod.ds[indices_of_outliers],
            prod.y[indices_of_outliers],
            'ro',
            markersize = 7,
            label='outliers')
        plt.legend(loc='best')

        save_file = os.path.join(dir_name,str(cus_no)+"_"+str(mat_no)+"_raw_data.png") 

        plt.savefig(save_file, bbox_inches='tight')
        plt.close(fig)

        # remove outlier
        prod.loc[indices_of_outliers,'y'] = None

        #plotting Data
        fig = plt.figure()
        plt.plot(prod.ds,prod.y)
        plt.xlabel('Date')
        plt.ylabel('Quantity')
        plt.legend()

        save_file = os.path.join(dir_name,str(cus_no)+"_"+str(mat_no)+"_clean_data.png") 

        plt.savefig(save_file, bbox_inches='tight')
        plt.close(fig)

        while(len(rem_data.ds) >= 2):

            #ARIMA Model Data Transform
            train_arima = train.set_index('ds',drop=True)
            # train_arima.head()
            test_arima = test.set_index('ds',drop= True)

            # ARIMA Model
            #grid search parameters
            p = d = q = range(0, 2)

            # Generate all different combinations of p, q and q triplets
            pdq = list(itertools.product(p, d, q))

            # Generate all different combinations of seasonal p, q and q triplets
            seasonal_pdq = [(x[0], x[1], x[2], 52) for x in list(itertools.product(p, d, q))]

            print('Next Test Starts...')

            # grid search
            warnings.filterwarnings("ignore") # specify to ignore warning messages
            min_aic = 9999999
            for param in pdq:
                for param_seasonal in seasonal_pdq:
                    try:
                        mod = sm.tsa.statespace.SARIMAX(train_arima,order=param,seasonal_order=param_seasonal,
                                                        enforce_stationarity=False,enforce_invertibility=False,measurement_error= True)

                        results = mod.fit()
                        if results.aic < min_aic:
                            min_aic = results.aic
                            opt_param = param
                            opt_param_seasonal = param_seasonal

#                         print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
                    except:
                        continue
            print('Optimal ARIMA{}x{}12 - AIC:{}'.format(opt_param, opt_param_seasonal, min_aic))

            #fitting Model
            mod = sm.tsa.statespace.SARIMAX(train_arima,order=opt_param,seasonal_order=opt_param_seasonal,
                                            enforce_stationarity=False,enforce_invertibility=False,measurement_error= True)
            result = mod.fit(disp=False)

            # forecast Train
            pred_train = results.get_prediction(start=pd.to_datetime(min(train_arima.index)), dynamic=False)
            pred_train_ci = pred_train.conf_int()

            # forecast test
            pred_test = results.get_prediction(start=pd.to_datetime(max(train_arima.index)),end=pd.to_datetime(max(test_arima.index)), dynamic=True)
            pred_test_ci = pred_test.conf_int()

            # ceating test and train emsembled result  
            #test result
            result_test = test
            result_test['y_ARIMA'] = np.array(pred_test.predicted_mean)[1:]

            # prophet
            m = Prophet(weekly_seasonality=False,yearly_seasonality=False,changepoint_prior_scale=5)
            m.fit(train);

            #creating pred train and test data frame 
            past = m.make_future_dataframe(periods=0, freq= 'W')
            future = pd.DataFrame(test['ds'])
            pf_train_pred = m.predict(past)
            pf_test_pred = m.predict(future)
            pf_train_pred = pf_train_pred[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].set_index([past.index])
            pf_test_pred = pf_test_pred[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].set_index([future.index])

            # ceating test and train emsembled result  
            #test result
            result_test['y_Prophet'] = np.array(pf_test_pred.yhat)

            #Ansemble 
            result_test['y_Ensembled'] = result_test[["y_ARIMA", "y_Prophet"]].mean(axis=1)

            train = prod[:(max(train.index)+1+test_points)]
            test = prod[(max(train.index)+1):(max(train.index)+1+test_points)]
            rem_data = prod[(max(train.index)+test_points):]

            output_result = pd.concat([output_result,result_test], axis=0)
            output_result['Error'] = np.subtract(output_result.y_Ensembled,output_result.y)
            output_result['Error_Cumcum'] = output_result.Error.cumsum()/output_result.y.cumsum()*100

        # Plot incremental test result
        fig = plt.figure()
        plt.plot(output_result.ds,output_result.y, label = 'Observed_test')
        plt.plot(output_result.ds,output_result.y_ARIMA, label = 'ARIMA')
        plt.plot(output_result.ds,output_result.y_Prophet, label = 'Prophet')
        plt.plot(output_result.ds,output_result.y_Ensembled, label = 'Ensembled')
        plt.xlabel('Date')
        plt.ylabel('Quantity')
        plt.legend()
        save_file = os.path.join(dir_name,str(cus_no)+"_"+str(mat_no)+"_prediction.png") 

        plt.savefig(save_file, bbox_inches='tight')
        plt.close(fig)

        #plot error
        fig = plt.figure()

        output_result['Error'] = np.subtract(output_result.y_Ensembled,output_result.y)
        output_result['Error_Cumsum'] = output_result.Error.cumsum()/output_result.y.cumsum()*100

        output_result['Error_prophet'] = np.subtract(output_result.y_Prophet,output_result.y)
        output_result['Error_Cumsum_prophet'] = output_result.Error_prophet.cumsum()/output_result.y.cumsum()*100

        output_result['Error_arima'] = np.subtract(output_result.y_ARIMA,output_result.y)
        output_result['Error_Cumsum_arima'] = output_result.Error_arima.cumsum()/output_result.y.cumsum()*100

        plt.plot(output_result.ds[2:], output_result.Error_Cumsum_arima[2:], label = 'ARIMA')
        plt.plot(output_result.ds[2:], output_result.Error_Cumsum_prophet[2:], label = 'Prophet')
        plt.plot(output_result.ds[2:], output_result.Error_Cumsum[2:], label = 'Ensembled')

        plt.xlabel('Date')
        plt.ylabel('% Cumulative Error')
        plt.legend()

        save_file = os.path.join(dir_name,str(cus_no)+"_"+str(mat_no)+"_cum_error.png") 

        plt.savefig(save_file, bbox_inches='tight')
        plt.close(fig)

Next Test Starts...
Optimal ARIMA(0, 0, 0)x(1, 1, 0, 52)12 - AIC:-12.524235893812964
Next Test Starts...
Optimal ARIMA(1, 0, 1)x(1, 1, 0, 52)12 - AIC:-0.9078664892756692
Next Test Starts...
Optimal ARIMA(1, 0, 0)x(1, 1, 0, 52)12 - AIC:17.781842670492587
Next Test Starts...
Optimal ARIMA(0, 1, 0)x(1, 1, 0, 52)12 - AIC:33.0702554145123
Next Test Starts...
Optimal ARIMA(0, 1, 0)x(1, 1, 0, 52)12 - AIC:49.899265618970446
Next Test Starts...
Optimal ARIMA(0, 1, 0)x(1, 1, 0, 52)12 - AIC:60.349434009304005
Next Test Starts...
Optimal ARIMA(1, 0, 0)x(1, 1, 0, 52)12 - AIC:73.40015375823569
Next Test Starts...
Optimal ARIMA(1, 0, 0)x(1, 1, 0, 52)12 - AIC:83.45664744554787
Next Test Starts...
Optimal ARIMA(0, 1, 0)x(1, 1, 0, 52)12 - AIC:94.90071975144548
Next Test Starts...
Optimal ARIMA(0, 0, 0)x(1, 1, 0, 52)12 - AIC:-15.90428575846187
Next Test Starts...
Optimal ARIMA(1, 0, 1)x(1, 1, 0, 52)12 - AIC:-6.098022230424782
Next Test Starts...
Optimal ARIMA(1, 1, 0)x(1, 1, 0, 52)12 - AIC:-11.0489558364

In [60]:
# # Error Arima
# abs(sum(output_result.y_ARIMA)-sum(output_result.y))/sum(output_result.y)*100

In [61]:
# # Error Prophet
# abs(sum(output_result.y_Prophet)-sum(output_result.y))/sum(output_result.y)*100

In [62]:
# # Error Ensembled
# abs(sum(output_result.y_Ansembled)-sum(output_result.y))/sum(output_result.y)*100

In [63]:
# data.matnr.unique()

array(['500008686', '500008702', '500008777', '500008822', '500008847',
       '500008848', '500008849', '500008851', '500008853', '500008857',
       '500008858', '500008859', '500008861', '500008862', '500008864',
       '500008868', '500008876', '500008901', '500008912', '500008921',
       '500008952', '500008966', '500008998', '500009000', '500009006',
       '500009030', '500009031', '500009032', '500009033', '500009065',
       '500009067', '500009117', '500009144', '500009204', '500009223',
       '500009225', '500009283', '500009290', '500009305', '500009311',
       '500009317', '500009340', '500009346', '500009424', '500009520',
       '500009627', '500009657', '500009664', '500009670', '500009707',
       '500009744', '500009750', '500009756', '500009877', '500009902',
       '500009903', '500009904', '500009908', '500009924', '500009996',
       '500009997', '500010003', '500010006', '500010010', '500010013',
       '500010015', '500010033', '500010034', '500010035', '5000

,customernumber,matnr,quantity,q_indep_p,dt_week
426928,500095627,100267,31.0,NaN,2015-02-26
426929,500095627,100267,7.0,NaN,2015-03-05
426930,500095627,100267,24.0,NaN,2015-03-12
426931,500095627,100267,37.0,NaN,2015-03-19
426932,500095627,100267,33.0,NaN,2015-03-26
